In [1]:
import os
os.chdir('/content/drive/MyDrive/000GithubRepos/bigcontest2024')

In [2]:
pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.2/255.2 kB 4.6 MB/s eta 0:00:00


In [3]:
pip install neo4j

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.6/296.6 kB 5.0 MB/s eta 0:00:00


In [4]:
pip install python-dotenv

In [5]:
from sentence_transformers import SentenceTransformer

from neo4j import GraphDatabase
from transformers import AutoTokenizer, AutoModel
import torch
import os
from tqdm import tqdm
from dotenv import load_dotenv, find_dotenv

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [6]:
load_dotenv(find_dotenv())

True

In [7]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [8]:
uri = os.environ["NEO4J_URI"]
username = os.environ["NEO4J_USERNAME"]
password = os.environ["NEO4J_PASSWORD"]

# Initialize the Neo4j driver
driver = GraphDatabase.driver(uri, auth=(username, password))

# Download from the 🤗 Hub
model = SentenceTransformer("upskyy/bge-m3-korean").to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/664 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [9]:
# Function to get embeddings for a given text
def get_embedding(text):
    text_embeddings = model.encode(text).squeeze().tolist()
    return text_embeddings

In [ ]:
with driver.session() as session:
    # Query to get all reviews
    query = "MATCH (r:Google) WHERE r.textEmbedding IS NULL RETURN r.id AS id, r.storePk AS storePk, r.text AS text"
    results = session.run(query)
    results_list = list(results)

In [ ]:
len(results_list)

27191

In [ ]:
def get_batch_embedding(text_lst):


In [ ]:

embeddings

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
with driver.session() as session:

    embeddings = model.encode([record['text'] for record in results_list[:5000]])
    for i, record in tqdm(enumerate(results_list[:5000])):
        review_id = record["id"]
        store_pk = record["storePk"]
        review_text = record["text"]

        update_query = """
        MATCH (r:Google {id: $review_id, storePk: $store_pk})
        SET r.textEmbedding = $embedding
        """
        session.run(update_query, review_id=review_id, store_pk=store_pk, embedding=embeddings[i])


KeyboardInterrupt: 

In [ ]:
# Function to update embeddings in Neo4j
def update_review_embeddings():
    with driver.session() as session:
        # Query to get all reviews
        query = "MATCH (r:Google) WHERE r.textEmbedding IS NULL RETURN r.id AS id, r.storePk AS storePk, r.text AS text"
        results = session.run(query)
        results_list = list(results)

        # Iterate through each review and update its embedding
        for record in tqdm(results_list, desc="Updating Embeddings") :
            review_id = record["id"]
            store_pk = record["storePk"]
            review_text = record["text"]

            # Generate the embedding for the review text
            embedding = get_embedding(review_text)

            # Update the Review node with the embedding
            update_query = """
            MATCH (r:Google {id: $review_id, storePk: $store_pk})
            SET r.textEmbedding = $embedding
            """
            session.run(update_query, review_id=review_id, store_pk=store_pk, embedding=embedding)

# Run the embedding update function
update_review_embeddings()

# Close the driver connection
driver.close()

Updating Embeddings:  10%|█         | 2554/25284 [32:29<4:49:10,  1.31it/s]


In [ ]:
driver.close()

NameError: name 'driver' is not defined